In [1]:
import re

In [2]:
re.match('abc', 'abcdef')

<_sre.SRE_Match object; span=(0, 3), match='abc'>

Fix spelling

plot word length in documents - corresponding with how many themes there are

- Focus on nouns or verbs
- Deidentify names

Naieve bayes   
SVM  
Logistic Regression   

Confusion Matrix

In [1]:
ls

01_aq_eda-qualitative-data.ipynb


In [8]:
ls ../data/raw

2018 WES Qual Coded - Final Comments and Codes for Martin Capstone_sample.xlsx*
2018 WES Qual Coded - Final Comments and Codes for Martin Capstone_sample2.xlsx
2018 WES Qual Coded - Final Comments and Codes.xlsx
WES 2007-2018 LONGITUDINAL DATA_sample-labeled.csv
WES 2007-2018 LONGITUDINAL DATA_sample.csv
WES 2007-2018 LONGITUDINAL DATA_sample.sav
WES 2007-2018 LONGITUDINAL DATA_short.sav


## EDA of text data

In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from pattern.en import suggest

In [217]:
from pattern.en import suggest

In [16]:
df_raw = pd.read_excel("../data/raw/2018 WES Qual Coded - Final Comments and Codes.xlsx",
                       skiprows = 1)


In [77]:
df_raw_test = df_raw.sample(frac = 0.1, random_state = 2019)
df_raw = df_raw.drop(index = df_raw_test.index)

In [85]:
pd.DataFrame()

(13379, 81)

In [87]:
df = df_raw[['2018 Comment']].join(df_raw.loc[:,'CPD':'OTH'])
df = df.rename(columns = {'2018 Comment' : 'comment'})

In [173]:
def correct_spelling(words):
    """Replace all misspelled tokenized words in a list with the correct spelling"""
    new_words = []
    for word in words:
        suggested_word = suggest(word)
        if suggested_word[0][1] > 0.75:
            new_word = suggested_word[0][0]
        else:
            new_word = word
        new_words.append(new_word)
    return new_words

In [168]:
X = df_raw.loc[:,"2018 Comment"]
Y = df_raw.loc[:,"CPD":"OTH"]

In [196]:
correct_spelling(test)

['fannnnnnnnntastik', 'screw', 'Aaron']

In [216]:
suggest("bitchh")

[('bitch', 1.0)]

In [207]:
def reduce_lengthening(words):
    """Reduce greater than 2 repeating characters in a word from list of tokenized words"""
    new_words = []
    for word in words:
        pattern = re.compile(r"(.)\1{2,}")
        new_word = pattern.sub(r"\1\1", word)
        new_words.append(new_word)
    return new_words

In [214]:
correct_spelling(reduce_lengthening(test))

['fantastic', 'bitch', 'unnessecary']

In [166]:
if new_word[0][1] > 0.75:
    new_word = suggested_word

0.43636363636363634

In [94]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y,
                                                      test_size = 0.25,
                                                      random_state = 2019)

In [219]:
test = X_train.values[80]

In [221]:
from nltk.tokenize import sent_tokenize



In [222]:
sent_tokenize(test)

["I think it's time to get rid of the behavioural event interview.",
 "One's work experience should not be limited to one or two events if one's career spans two decades.",
 'That is not a very accurate way go gauge a persons suitability for a job.',
 'What ever happened to promotion from within?',
 'If someone has been doing a good job for several years, they should be rewarded with a promotion.',
 'This happens all the time in industry.',
 'Why not in government as well?',
 'The reclassification process was not a success.',
 'Many staff doing similar work are classified differently.',
 'In addition, I know of several supervisors who are classified at the same level as entry level staff.']

In [75]:

df = df_raw[['2018 Comment']].join(df_raw.loc[:,'CPD':'OTH'])
df = df.rename(columns = {'2018 Comment' : 'comment'})


df['count_themes'] = df.drop(columns='comment').sum(axis = 1)
df['count_words'] = [len(s.split()) for s in df.comment]

In [76]:
df.head()

,comment,CPD,CB,EWC,Exec,FWE,SP,RE,Sup,SW,TEPE,VMG,OTH,count_themes,count_words
0,I would suggest having a developmental growth ...,1,0,0,0,0,1,0,0,0,0,0,0,2,70
1,Base decisions regarding fish and wildlife on ...,0,0,0,0,0,0,0,0,0,0,1,0,1,14
2,The problem with the BCSS is Linda Cavanaugh a...,0,0,0,1,0,0,1,0,0,0,0,0,2,134
3,"Improved office space (fix HVAC, etc) but NO LWS",0,0,0,0,1,0,0,0,0,1,0,0,2,9
4,Get rid of Leading Workplace Strategies and gi...,0,0,0,0,1,0,0,0,0,0,0,0,1,12


In [223]:
import spacy

nlp = spacy.load("en_core_web_sm")


In [74]:
doc = nlp(u"Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [242]:
docs = [nlp(comment) for comment in X_train]

In [270]:
doc = nlp(X_train.values[10])

In [271]:
doc.text

'I would like to know more about coming job opportunities and maybe get some information on how to win those competitions.'

In [241]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

222 28 31 CARDINAL
Main 32 36 PERSON
Half 53 57 CARDINAL
222 301 304 CARDINAL
our day to day 341 355 DATE


In [265]:
documents = []
person_text = []
start_char = []
end_char = []

for doc in docs:
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            documents.append(doc.text)
            person_text.append(ent.text)
            start_char.append(ent.start_char)
            end_char.append(ent.end_char)

df_persons = pd.DataFrame({'documents': documents, 
                          'person_text': person_text,
                          'start_char': start_char,
                          'end_char': end_char})

df_persons

,documents,person_text,start_char,end_char
0,Greater number of AND greater frequency of (ev...,Temporary Assignments,96,117
1,Knowledgebase MUST be updated more timely - we...,Topic Guides,239,251
2,The physical environment at 222 Main is disgus...,Main,32,36
3,I think there is tremendous top down leadershi...,John Smith,254,264
4,Physical space in Mcfd is in need of repair. C...,Mcfd,18,22
5,Need more evidence-based strategic decision ma...,Directors,477,486
6,Lower and Middle management need to have stren...,Middle,10,16
7,It would be great if we have more frequent car...,Dirty,410,415
8,Valuing the operations side of our organizatio...,Lori Halls,580,590
9,The only thing that matters to myself and my c...,Sheriffs,109,117


In [293]:
df_persons.documents[633]

'The succession planning in this building in the past has been driven by self serving, ego centric, unintelligent sociopaths. Scott Dempsey, Vicky Davis and Ted Leonard are leftovers from a management team who made it their priority to control this place by fear. This building should be a place of healing, not corrupted by the unkind actions of those who seek to only serve themselves. Staff and inmates are both suffering, we all lack a safe environment to exist in, and have to worry about the predatory behaviours of managers abusing their power. Kindness should be celebrated, promoted and cultured amongst us all, that direction starts from above, provide us with social models we can emulate and follow into the future. We are relying on you.'

In [266]:
df_persons.sort_values('person_text')

,documents,person_text,start_char,end_char
367,The managers at the ADW level at VIRCC are ver...,ADW Davis,260,269
38,"I am a single income, I cannot afford to live ...",AETP,489,493
444,The ICM database system is mind-blowingly limi...,ASAP,309,315
510,Crown needs to be in the courthouse. Abbotsfor...,Abbotsford,37,47
250,Better communication from the field to provinc...,Aboriginal,205,215
327,Team days that are not simply educational lect...,Aboriginal,179,189
187,"Help us address our need for space, open anoth...",Aboriginal,538,548
197,I would recommend that some Branches have more...,Aboriginal,163,173
248,EDs and DoOs should have the responsibility to...,Aboriginal,434,444
191,I would like more learning added to the Learni...,Acrobrat,334,342


In [291]:
import human

ModuleNotFoundError: No module named 'human'

In [288]:
names.

'David'

In [285]:
df_persons.person_text.sort_values().unique()

array(['ADW Davis', 'AETP', 'ASAP  ', 'Abbotsford', 'Aboriginal',
       'Acrobrat', 'Admin', "Admin Officer's", 'Admin Staff', 'Adobe',
       'Adult', 'Agenda', 'Agile', 'Alex Fraser', 'Alice', 'Alison Bond',
       'Allow', "Altern've", 'Angler Creel', 'Ann Hart', 'Appropriate',
       'Approval', 'ArcGIS Pro', 'Assignments', 'Associate', 'Attendance',
       'Aware', 'Axiom', 'B.C.  ', 'BCTS', 'Band 2', 'Band 3',
       'Band Positions', 'Barry', 'Becoming Lean', 'Bi-Yearly',
       'Blanshard St', 'Bob Chartier', 'Bob Turner', 'Bobbi', 'Branch,',
       'Branches', 'Build', 'Burnaby', 'Bus Ops', 'Byron Jonson',
       'Calendars', 'Camosun', 'Cap', 'Captains', 'Care Plans',
       'Case Managers', 'Centralized Hiring', 'Chad Hoskins', 'Chairs',
       'Charlie South', 'Charlie West', 'Cheryl', 'Cheryl  ', 'Child',
       'Christopherson', 'Cindy Kukucska', 'Clarify', 'Classification',
       'Clerical', 'Cloud Strategy', 'Code 6', 'Competitive Salary',
       'Complete MyPerforman